In [1]:
import sys
import arcpy
sys.path.append("../src")
from utils import Utils, WorkspaceCleaner, demUtils
from landforms import Landforms
from derivatives import HabitatDerivatives
from extents import GetCoordinates
from inpainter import Inpainter
import os
import glob

# 1. Indexing the Tiff files

In [ ]:
## Manual input of the DEM to be used
input_dem = r"../dem/dem.tif"
# input_dem = r"dem\Ingalls Point 1m_clipped.tif"
# Define the products to be created
# 
products = ["slope", "aspect", "roughness", "tpi", "tri", "hillshade"]
products = ["shannon_index"]
# products = ["slope"]
print("ORIGINAL PATH:", input_dem)
print("PRODUCTS:", products)
GetCoordinates().return_min_max_tif_df([input_dem])

# 2. Generate derived DEM products

In [ ]:
# Generate output file paths
# Define the output folder path for habitat derivates
out_folder = "..\\habitat_derivatives"
# Create an instance of the HabitatDerivatives class with the specified parameters
habitat_derivatives = HabitatDerivatives(
                                        input_dem=input_dem, 
                                        output_folder=out_folder,
                                        products=products,
                                        shannon_window=9,
                                        fill_iterations=1,
                                        fill_method=None,
                                        chunk_size=32,
                                        generate_boundary=True
                                        )
habitat_derivatives.process_dem()

DEM NAME: dem
OUTPUT FOLDER: ..\habitat_derivatives\dem
  Successfully deleted Temporary Workspace: C:\Users\ageglio\AppData\Local\Temp\2\tmpfd665toz   
  Successfully deleted Local Temporary Workspace: ..\habitat_derivatives\dem\temp            
Temporary file cleanup process completed successfully.                           
Generating products ['shannon_index']


In [ ]:
# Trim the backscatter (bs) to the extent of the DEM
if 'input_bs' in locals() and input_bs:
    print("TRIMMING BACKSCATTER TO DEM EXTENT")
    # intitialize the Inpainter class with the input DEM and output folder
    inpainter = Inpainter(input_dem, save_path=out_folder)
    # trim the backscatter raster using the binary mask
    binary_mask, dissolved_polygon = inpainter.get_data_boundary()
    out_bs = inpainter.trim_raster(input_bs, binary_mask, overwrite=False)
    # Clean up the workspace
    WorkspaceCleaner(inpainter).clean_up()
    print("TRIMMED BACKSCATTER:", out_bs)
else:
    print("no backscatter provided")

## 3. Create geomorphons landforms using arcpy 3.4

In [ ]:
# create original landforms from ArcGIS Pro
landforms = Landforms(input_dem)
landforms.calculate_geomorphon_landforms()

# calculate the 10class solution
output_file10c = landforms.classify_bathymorphons(classes="10c")
print(f"Modified raster data saved to {output_file10c}")

# # calculate the 6class solution
output_file6c = landforms.classify_bathymorphons(classes="6c")
print(f"Modified raster data saved to {output_file6c}")

# # calculate the 5class solution
output_file5c = landforms.classify_bathymorphons(classes="5c")
print(f"Modified raster data saved to {output_file5c}")

# # calculate the 4class solution
output_file4c = landforms.classify_bathymorphons(classes="4c")
print(f"Modified raster data saved to {output_file4c}")

In [ ]:
## If you want to save the landforms classification histogram and metadata
Landforms.analyze_raster_data(output_file6c)

# 4. Tracklines to shapefile

In [ ]:
# allowing the user to run all without executing the trackline portion
assert 1==2

In [ ]:
from shapely.geometry import LineString, Point, Polygon
import geopandas as gpd

In [ ]:
# Input path to tracklines folders from Qimera
tracklines_folders_paths = r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Tracks_Data_Release_NLMI_Reefs\Tracklines\*\*"
tracklines_folders = glob.glob(tracklines_folders_paths)

# Define the output folder path for shapefiles
out_folder = "shapefiles"
os.makedirs(out_folder, exist_ok=True)

trackline_folder = tracklines_folders[4] #<------------------------ CHANGE INDEX ID
print("folder path chosen: ", trackline_folder)

## 3b. combine and convert tracklines to a shapefile

In [ ]:
def create_tracklines(trackline_folder):
    tracklines_files = glob.glob(os.path.join(trackline_folder, "*.txt"))
    reef_name = os.path.basename(trackline_folder)
    print("creating tracklines shapefile for: ", reef_name)
    output_shapefile_folder = os.path.join(out_folder, reef_name)
    os.makedirs(output_shapefile_folder, exist_ok=True)
    output_shapefile = os.path.join(output_shapefile_folder, f"{reef_name}.shp")

    # Concatenate all dataframes in the list into a single dataframe
    tracklines = pd.concat([pd.read_csv(file, delimiter=',', header=None) for file in tracklines_files], ignore_index=True)
    tracklines.columns = ["UTC", "X", "Y", "Delta"]

    # Get the lat lon coordinates of the raw combined tracklines file
    xy = GetCoordinates.convert_tracklines_to_lat_lon(tracklines, from_wkt="wgs84_16N_OGC_WKT.txt", wgs84_wkt="wgs84_OGC_WKT.txt")

    # Create a GeoDataFrame and save to a shapefile
    gpd.GeoDataFrame(geometry=[LineString(xy)], crs="EPSG:4326").to_file(output_shapefile)

create_tracklines(trackline_folder)
